In [1]:
import chess

PIECE_VALUE = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0  # Il valore del re è gestito separatamente
}

# Tabelle di posizione per il pedone
PAWN_TABLE = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, -5, -10, 0, 0, -10, -5, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per il cavallo
KNIGHT_TABLE = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20, 0, 5, 5, 0, -20, -40,
    -30, 5, 10, 15, 15, 10, 5, -30,
    -30, 0, 15, 20, 20, 15, 0, -30,
    -30, 5, 15, 20, 20, 15, 5, -30,
    -30, 0, 10, 15, 15, 10, 0, -30,
    -40, -20, 0, 0, 0, 0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50,
]

# Tabelle di posizione per l'alfiere
BISHOP_TABLE = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10, 5, 0, 0, 0, 0, 5, -10,
    -10, 10, 10, 10, 10, 10, 10, -10,
    -10, 0, 10, 10, 10, 10, 0, -10,
    -10, 5, 5, 10, 10, 5, 5, -10,
    -10, 0, 5, 10, 10, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20
]

# Tabelle di posizione per la torre
ROOK_TABLE = [
    0, 0, 0, 5, 5, 0, 0, 0,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    5, 10, 10, 10, 10, 10, 10, 5,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per la regina
QUEEEN_TABLE = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10, 0, 5, 0, 0, 0, 0, -10,
    -10, 5, 5, 5, 5, 5, 0, -10,
    0, 0, 5, 5, 5, 5, 0, -5,
    -5, 0, 5, 5, 5, 5, 0, -5,
    -10, 0, 5, 5, 5, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20
]

# Tabelle di posizione per il re (inizio gioco)
KING_INITGAME_TABLE = [
    20, 30, 10, 0, 0, 10, 30, 20,
    20, 20, 0, 0, 0, 0, 20, 20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30
]

# Tabelle di posizione per il re (fine gioco)
KING_ENDGAME_TABLE = [
    -50, -40, -30, -20, -20, -30, -40, -50,
    -30, -20, -10, 0, 0, -10, -20, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -30, 0, 0, 0, 0, -30, -30,
    -50, -30, -30, -30, -30, -30, -30, -50
]

In [2]:
class EvaluatePiecePositions:

    def evaluate_piece_positions(self, board, piece_table, piece_type, color):
        score = 0
        pieces = board.pieces(piece_type, color)
        for square in pieces:
            piece_position_value = piece_table[square]
            score += piece_position_value if color == chess.WHITE else -piece_position_value
        return score

    def is_endgame(self, board):
        # Regine
        white_queens = len(board.pieces(chess.QUEEN, chess.WHITE))
        black_queens = len(board.pieces(chess.QUEEN, chess.BLACK))

        # se entrambi i lati non hanno Regine -> endgame phase
        if white_queens == 0 and black_queens == 0:
            return True

        # Minorpieces
        white_bishops = len(board.pieces(chess.BISHOP, chess.WHITE))
        black_bishops = len(board.pieces(chess.BISHOP, chess.BLACK))
        white_knights = len(board.pieces(chess.KNIGHT, chess.WHITE))
        black_knights = len(board.pieces(chess.KNIGHT, chess.BLACK))
        white_minors = white_bishops + white_knights
        black_minors = black_bishops + black_knights

        white_rooks = len(board.pieces(chess.ROOK, chess.WHITE))
        black_rooks = len(board.pieces(chess.ROOK, chess.BLACK))

        # se ogni lato che ha una regina, non ha altri pezzi oppure ha
        # 1 Minorpiece al massimo -> endgame phase
        # fmt: off
        white_endgame_condition_with_queen = (
                white_queens == 1 and (white_rooks == 0 and white_minors <= 1)
        )
        black_endgame_condition_with_queen = (
                black_queens == 1 and (black_rooks == 0 and black_minors <= 1)
        )
        # fmt: on

        if (
                (white_endgame_condition_with_queen and black_queens == 0)
                or (black_endgame_condition_with_queen and white_queens == 0)
                or (
                white_endgame_condition_with_queen
                and black_endgame_condition_with_queen
        )
        ):
            return True

        return False

    def h(self, board):
        total_score = 0
        if self.is_endgame(board):
            king_table_to_use = KING_ENDGAME_TABLE
        else:
            king_table_to_use = KING_INITGAME_TABLE
        total_score += self.evaluate_piece_positions(board, PAWN_TABLE, chess.PAWN, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, KNIGHT_TABLE, chess.KNIGHT, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, BISHOP_TABLE, chess.BISHOP, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, ROOK_TABLE, chess.ROOK, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, QUEEEN_TABLE, chess.QUEEN, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, king_table_to_use, chess.KING, chess.WHITE)

        total_score -= self.evaluate_piece_positions(board, PAWN_TABLE, chess.PAWN, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, KNIGHT_TABLE, chess.KNIGHT, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, BISHOP_TABLE, chess.BISHOP, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, ROOK_TABLE, chess.ROOK, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, QUEEEN_TABLE, chess.QUEEN, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, king_table_to_use, chess.KING, chess.BLACK)

        return total_score


class EvaluatePiecePositions1:

    def evaluate_piece_positions(self, board, piece_table, piece_type, color):
        score = 0
        pieces = board.pieces(piece_type, color)
        for square in pieces:
            piece_position_value = piece_table[square]
            score += piece_position_value if color == chess.WHITE else -piece_position_value
        return score

    def is_endgame(self, board):
        # Limite del valore complessivo dei pezzi per considerare una partita nel finale
        endgame_value_threshold = 13

        total_value = 0
        for piece_type in PIECE_VALUE:
            total_value += len(board.pieces(piece_type, chess.WHITE)) * PIECE_VALUE[piece_type]
            total_value += len(board.pieces(piece_type, chess.BLACK)) * PIECE_VALUE[piece_type]

        # Se il valore totale dei pezzi, esclusi i re, è al di sotto di una certa soglia,
        # consideriamo la partita come nel finale.
        is_endgame_phase = total_value <= endgame_value_threshold

        # Ulteriori considerazioni potrebbero essere la presenza di molti pedoni, il che potrebbe
        # suggerire che non siamo ancora nel finale tipico, anche se il valore dei pezzi è basso.
        if len(board.pieces(chess.PAWN, chess.WHITE)) + len(board.pieces(chess.PAWN, chess.BLACK)) > 10:
            is_endgame_phase = False

        return is_endgame_phase

    # Funzione per calcolare il punteggio totale basato sulla posizione dei pezzi
    def h(self, board):
        total_score = 0
        if self.is_endgame(board):
            king_table_to_use = KING_ENDGAME_TABLE
        else:
            king_table_to_use = KING_INITGAME_TABLE
        total_score += self.evaluate_piece_positions(board, PAWN_TABLE, chess.PAWN, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, KNIGHT_TABLE, chess.KNIGHT, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, BISHOP_TABLE, chess.BISHOP, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, ROOK_TABLE, chess.ROOK, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, QUEEEN_TABLE, chess.QUEEN, chess.WHITE)
        total_score += self.evaluate_piece_positions(board, king_table_to_use, chess.KING, chess.WHITE)

        total_score -= self.evaluate_piece_positions(board, PAWN_TABLE, chess.PAWN, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, KNIGHT_TABLE, chess.KNIGHT, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, BISHOP_TABLE, chess.BISHOP, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, ROOK_TABLE, chess.ROOK, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, QUEEEN_TABLE, chess.QUEEN, chess.BLACK)
        total_score -= self.evaluate_piece_positions(board, king_table_to_use, chess.KING, chess.BLACK)

        return total_score



In [3]:
from itertools import islice

board = chess.Board(fen="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
moves = board.legal_moves
evaluation = EvaluatePiecePositions1()
evaluation1 = EvaluatePiecePositions()
# evaluation2 = EvaluateMobility2()
print(board)
print(evaluation.h(board))
print(evaluation1.h(board))
# print(evaluation2.h(board))
move = next(islice(moves, 10, 11), None)
board.push(move)
moves = board.legal_moves
print(board)
print(evaluation.h(board))
# print(evaluation1.h(board))
# print(evaluation2.h(board))
for i in range(200):
    moves = board.legal_moves
    if board.turn:
        k = moves.count()
        move = next(islice(moves, k - 1, k), None)
    else:
        k = moves.count()
        move = next(islice(moves, k - 2, k), None)
    board.push(move)
    print(board.turn)
    print(board)
    print(evaluation.h(board))
    print(evaluation1.h(board))
#    print(evaluation1.h(board))
#     print(evaluation2.h(board))
%timeit evaluation.h(board)
%timeit evaluation1.h(board)
#  %timeit evaluation2.h(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
150
150
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. P . . . . . .
P . P P P P P P
R N B Q K B N R
135
True
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
. . . . . . . .
. P . . . . . .
P . P P P P P P
R N B Q K B N R
90
90
False
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
P . . . . . . .
. P . . . . . .
. . P P P P P P
R N B Q K B N R
85
85
True
r n b q k b n r
p . . p p p p p
. . . . . . . .
. p p . . . . .
P . . . . . . .
. P . . . . . .
. . P P P P P P
R N B Q K B N R
45
45
False
r n b q k b n r
p . . p p p p p
. . . . . . . .
. p p . . . . .
P . P . . . . .
. P . . . . . .
. . . P P P P P
R N B Q K B N R
35
35
True
r n b q k b n r
p . . . p p p p
. . . . . . . .
. p p p . . . .
P . P . . . . .
. P . . . . . .
. . . P P P P P
R N B Q K B N R
10
10
False
r n b q k b n r
p . . . p p p